In [ ]:
# Parameters and utilities
from pathlib import Path
import os
import sys
import subprocess
from typing import Dict, List, Union

# Project root and venv python
try:
    ROOT = Path(__file__).resolve().parent.parent
except NameError:
    # In Colab/Jupyter, __file__ is not defined; use current working directory
    ROOT = Path.cwd()
PY = ROOT/".venv"/"bin"/"python"
PYTHON = str(PY if PY.exists() else sys.executable)

# Common args placed here (do not hardcode inside functions)
EPOCHS = 100
PATIENCE = 10
BATCH_SIZE = 32
NUM_WORKERS = max(0, (os.cpu_count() or 2)//2)
# Optional dataset size overrides for geom_simple
TRAIN_SIZE_OVERRIDE = None  # e.g., 5000 or None
VAL_SIZE_OVERRIDE = None    # e.g., 200 or None
TEST_SIZE_OVERRIDE = None   # e.g., 200 or None

# Helper to build a training command
def build_cmd(name: str, data: str, repr_type: str, loss: str, extra: Dict[str, Union[str, float, int]] | None = None) -> List[str]:
    cmd = [
        PYTHON, str(ROOT/"experiments_refactored"/"train.py"),
        "--data", data,
        "--repr", repr_type,
        "--loss", loss,
        "--exp_name", name,
        "--epochs", str(EPOCHS),
        "--patience", str(PATIENCE),
        "--batch_size", str(BATCH_SIZE),
        "--num_workers", str(NUM_WORKERS),
        "--generate_data",
    ]
    if TRAIN_SIZE_OVERRIDE is not None:
        cmd += ["--train_size", str(TRAIN_SIZE_OVERRIDE)]
    if VAL_SIZE_OVERRIDE is not None:
        cmd += ["--val_size", str(VAL_SIZE_OVERRIDE)]
    if TEST_SIZE_OVERRIDE is not None:
        cmd += ["--test_size", str(TEST_SIZE_OVERRIDE)]

    # Resume from best checkpoint under experiments_refactored/results/<name>/
    best_ckpt = ROOT/"experiments_refactored"/"results"/name/"best_checkpoint.pth"
    if best_ckpt.exists():
        cmd += ["--resume", str(best_ckpt)]

    if extra:
        for k, v in extra.items():
            cmd += [f"--{k}", str(v)]
    return cmd

# Runner
def run_experiment(name: str, data: str, repr_type: str, loss: str, extra: Dict[str, Union[str, float, int]] | None = None) -> int:
    out_dir = ROOT/"experiments_refactored"/"results"/name
    out_dir.mkdir(parents=True, exist_ok=True)
    log_path = out_dir/"train.log"
    cmd = build_cmd(name, data, repr_type, loss, extra)
    print(" ".join(cmd))
    with log_path.open("a") as f:
        f.write(" ".join(cmd) + "\n\n")
        f.flush()
        proc = subprocess.run(cmd, stdout=f, stderr=f)
    print(f"Done: {name} status={proc.returncode}")
    return proc.returncode



In [ ]:
# Experiment 1: geom_pixel_DiceBCE
run_experiment(
    name="geom_pixel_DiceBCE",
    data="geom_simple",
    repr_type="pixel",
    loss="dice_bce",
    extra={}
)


In [ ]:
# Experiment 2: geom_pixel_FocalBCE
run_experiment(
    name="geom_pixel_FocalBCE",
    data="geom_simple",
    repr_type="pixel",
    loss="focal_bce",
    extra={}
)


In [ ]:
# Experiment 3: geom_pixel_WeightedBCE
run_experiment(
    name="geom_pixel_WeightedBCE",
    data="geom_simple",
    repr_type="pixel",
    loss="weighted_bce",
    extra={"pos_weight": 2000.0}
)


In [ ]:
# Experiment 4: geom_heat_WeightedMSE200
run_experiment(
    name="geom_heat_WeightedMSE200",
    data="geom_simple",
    repr_type="heat",
    loss="weighted_mse",
    extra={"pos_weight": 200.0}
)
